In [1]:
import cv2
import time 
import numpy as np 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import HandTracking as htm 
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

D:\ANACONDA\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\ANACONDA\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
D:\ANACONDA\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [2]:
# w,h=1100,480
cap=cv2.VideoCapture(0)
# cap.set(3,w)
# cap.set(4,h)

detector=htm.handDetector(detectionCon=0.9)   #detection confidence= 90%
#-----------------------------------------------
# Audio Control
#-----------------------------------------------

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
# volume.GetMute()
# volume.GetMasterVolumeLevel()
volrange= volume.GetVolumeRange()


minvol=volrange[0]
maxvol=volrange[1]


#------------------------------------------------
while True:
    ret,img=cap.read()
    img=detector.findHands(img)
    lmlist=detector.findPosition(img,draw=False)
    if len(lmlist)!=0:
        x1,y1=lmlist[4][1],lmlist[4][2]
        x2,y2=lmlist[8][1],lmlist[8][2]

        cx,cy= (x1+x2)//2, (y1+y2)//2      # Center of line

        cv2.circle(img, (x1,y1), radius=10, color=(255,0,0), thickness=-1)
        cv2.circle(img, (x2,y2), radius=10, color=(255,0,0), thickness=-1)
        cv2.line(img, (x1,y1), (x2,y2), color=(255,0,255), thickness=3)
        cv2.circle(img, (cx,cy), radius=6, color=(255,0,0), thickness=-1)

        length=math.hypot(x2-x1, y2-y1)
        # print(length)

        if length<50:
            cv2.circle(img, (cx,cy), radius=6, color=(0,255,0), thickness=-1)

        #Hand Range : 25- 215
        #Vol Range: -60  - 0

        vol=np.interp(length, [17,182], [minvol,maxvol])

        volBar =np.interp (length, [17,182], [400,150])

        volPer=np.interp (length, [17,182], [0,100])
        #print(int(length), vol )
        volume.SetMasterVolumeLevel(vol, None)

        cv2.rectangle(img, (50,150), (85,400), color=(255,0,255), thickness=3)
        cv2.rectangle(img, (50,int(volBar)), (85,400), color=(255,0,0), thickness=-1)
        # cv2.putText(img, f'{int(volPer)} %', (40,450), cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0), 2)

    cv2.imshow('Video',img)
    if cv2.waitKey(1) & 0XFF==27:
        break
cap.release()
cv2.destroyAllWindows()